# Testing that Python libraries are available on the Hub

In [ ]:
import pytest
import ipytest


ipytest.autoconfig()

## Data vis

### Plotly

In [ ]:
def test_plotly_create_simple_barchart():
    import plotly.graph_objects as go
    fig = go.Figure(
        data=[go.Bar(y=[2, 1, 3])],
        layout_title_text="A Figure Displayed with fig.show()"
    )
    fig.show()

### Widgets

In [ ]:
def test_interact_decorate_function():
    from ipywidgets import interact

    @interact
    def add(x=2.0, y=3.0):
        return x + y

### Matplotlib

In [ ]:
def test_matplotlib_simple_line_plot():
    import matplotlib.pyplot as plt


    plt.plot(range(10))

### Dask and visualizing dask arrays

In [ ]:
def test_dask_visualize():
    import dask.array as da

    x, y = da.random.uniform(size=(2, 10**5), chunks=10**4)
    pi = da.mean(x**2 + y**2 < 1) * 4

    pi.visualize()

### Altair and `vega_datasets`

In [ ]:
def test_altair_basic_scatterplot():
    import altair as alt
    from vega_datasets import data

    source = data.cars()

    brush = alt.selection(type='interval')

    points = alt.Chart(source).mark_point().encode(
        x='Horsepower:Q',
        y='Miles_per_Gallon:Q',
        color=alt.condition(brush, 'Origin:N', alt.value('lightgray'))
    ).add_selection(
        brush
    )

    bars = alt.Chart(source).mark_bar().encode(
        y='Origin:N',
        color='Origin:N',
        x='count(Origin):Q'
    ).transform_filter(
        brush
    )

    points & bars

### Datashader

In [ ]:
@pytest.fixture(scope='session')
def ny_taxi_points():
    from urllib.request import urlretrieve

    urlretrieve('https://github.com/holoviz/datashader/blob/master/examples/data/.data_stubs/nyc_taxi.csv?raw=true',
                'nyc_taxi.csv')
    import pandas as pd

    return pd.read_csv('nyc_taxi.csv', usecols=['dropoff_x', 'dropoff_y'])

In [ ]:
def test_datashader_ny_taxi_plot(ny_taxi_points):
    import datashader


    import datashader as ds
    from colorcet import fire
    from datashader import transfer_functions as tf

    agg = ds.Canvas().points(ny_taxi_points, 'dropoff_x', 'dropoff_y')
    tf.set_background(tf.shade(agg, cmap=fire),"black")

### Holoviews

In [ ]:
def test_holoviews_ny_taxi_plot(ny_taxi_points):
    import holoviews as hv
    from holoviews.element.tiles import EsriImagery
    from holoviews.operation.datashader import datashade
    hv.extension('bokeh')

    map_tiles  = EsriImagery().opts(alpha=0.5, width=900, height=480, bgcolor='black')
    points     = hv.Points(ny_taxi_points, ['dropoff_x', 'dropoff_y'])
    taxi_trips = datashade(points, x_sampling=1, y_sampling=1, cmap='fire', width=900, height=480)

    map_tiles * taxi_trips

### Network analysis

In [ ]:
def test_networkx_graphviz_plot():
    import matplotlib.pyplot as plt
    import networkx as nx

    try:
        import pygraphviz
        from networkx.drawing.nx_agraph import graphviz_layout
    except ImportError:
        try:
            import pydot
            from networkx.drawing.nx_pydot import graphviz_layout
        except ImportError:
            raise ImportError("This example needs Graphviz and either "
                              "PyGraphviz or pydot")

    G = nx.balanced_tree(3, 5)
    pos = graphviz_layout(G, prog='twopi')
    plt.figure(figsize=(8, 8))
    nx.draw(G, pos, node_size=20, alpha=0.5, node_color="blue", with_labels=False)
    plt.axis('equal')
    plt.show()

### Xarray

In [ ]:
@pytest.mark.skip(reason='Cartopy is a POS and crashes my kernal...')
def test_xarray_plot_with_cartopy_projection():
    import xarray as xr

    import cartopy.crs as ccrs
    import matplotlib.pyplot as plt

    url = 'https://github.com/mapbox/rasterio/raw/master/tests/data/RGB.byte.tif'
    da = xr.open_rasterio(url)

    # The data is in UTM projection. We have to set it manually until
    # https://github.com/SciTools/cartopy/issues/813 is implemented
    crs = ccrs.UTM('18N')

    # Plot on a map
    ax = plt.subplot(projection=crs)
    da.plot.imshow(ax=ax, rgb='band', transform=crs)
    ax.coastlines('10m', color='r')

### Generic visualisation imports

In [ ]:
data_viz_generic_imports = ['voila', 'dash']

@pytest.mark.parametrize(
    "library_name", 
    data_viz_generic_imports
)
def test_data_viz_generic_imports(library_name):
    import importlib
    
    importlib.import_module(library_name)

### Flask, SQLAlchemy, Flask-SQLAlchemy, Flask-Connexion

In [ ]:
flask_generic_imports = [
    'flask', 'sqlalchemy', 'flask_sqlalchemy', 'alembic', 'connexion', 'yaml', 'flask_apscheduler', 'jinja2',
    'itsdangerous'
]

### Database engines

In [ ]:
@pytest.mark.parametrize(
    'library_name',
    [pytest.param(name, id='flask') for name in flask_generic_imports]
)
def test_web_generic_imports(library_name):
    import importlib
    
    importlib.import_module(library_name)

In [ ]:
database_engines_basic = ['psycopg2', 'cx_Oracle', 'sqlite3', 'pymongo',
                        'mysql.connector']

In [ ]:
@pytest.mark.parametrize(
    'library_name',
    [pytest.param(name, id='engine') for name in database_engines_basic]
)
def test_db_basic_imports(library_name):
    import importlib
    
    importlib.import_module(library_name)

In [ ]:
more_database_engines = ['MySQLdb', 'pyodbc', 'teradatasql']

In [ ]:
@pytest.mark.parametrize(
    'library_name',
    [pytest.param(name, id='engine') for name in more_database_engines]
)
def test_more_db_imports(library_name):
    import importlib
    
    importlib.import_module(library_name)

In [ ]:
def test_redis_available():
    import redis

In [ ]:
def test_redis():
    # Likely to have problems with ports inside containers
    import redis
    r = redis.Redis()  # do we need / want to setup redis in the cluster?
    r.mset({"Croatia": "Zagreb", "Bahamas": "Nassau"})
    assert r.get("Bahamas") == b'Nassau'

import timescaledb  <- not a Python library

## Best Practices

In [ ]:
best_practices_libs = ['pylint', 'flake8', 'black', 
                       'pycodestyle', # formerly pep8
                       'hypothesis', 'pytest', 'coverage', 'logzero', 'inflect', 'arrow']


In [ ]:
@pytest.mark.parametrize(
    "library_name", 
    best_practices_libs
)
def test_best_practices_imports(library_name):
    import importlib
    
    importlib.import_module(library_name)

### Security

In [ ]:
security_libs = ['keyring', 'keyrings.cryptfile', 'passlib', 'bcrypt', 'correcthorse', 'watchdog']

In [ ]:
@pytest.mark.parametrize(
    "library_name", 
    security_libs
)
def test_security_imports(library_name):
    import importlib
    
    importlib.import_module(library_name)

## Data analysis and scientific computing

### HDF

In [ ]:
def test_pandas_read_hdf():
    import pandas as pd
    from os import path
    
    # hopefully the file is in the container already
    h5_path = '/Data/AAPL.h5' if path.exists('/Data') else '/tmp/AAPL.h5'
    apple = pd.read_hdf(h5_path)

    apple[:3]

def test_h5py_read_hdf():
    import h5py
    from os import path
    
    h5_path = '/Data/AAPL.h5' if path.exists('/Data') else '/tmp/AAPL.h5'

    apple_raw = h5py.File(h5_path, mode='r')

    apple_raw['AAPL'].keys()

### Image processing

In [ ]:
image_processing_libs = ['skimage', 'rasterio']

In [ ]:
@pytest.mark.parametrize(
    "library_name", 
    image_processing_libs
)
def test_image_processing_imports(library_name):
    import importlib
    
    importlib.import_module(library_name)

### ML

In [ ]:
ml_libs = ['sklearn', 'eli5', 'yellowbrick', 'textdistance', 'jellyfish']

# generally desirable ML libraries
# other_ml_libs = ['xgboost', 'keras', 'pytorch', 'tensorflow', ]

# compromise (pytorch cpu)
other_ml_libs = ['xgboost', 'torch', 'torchvision']

distributed_libs = ['distributed']

In [ ]:
@pytest.mark.parametrize(
    'library_name',
    [pytest.param(name, id='ml') for name in ml_libs] +
    [pytest.param(name, id='other_ml') for name in other_ml_libs] +
    [pytest.param(name, id='distributed') for name in distributed_libs]
)
def test_ml_processing_imports(library_name):
    import importlib
    
    importlib.import_module(library_name)

### IPython

In [ ]:
def test_ipython_display_image_from_url():
    from IPython.display import Image

    Image('https://imgs.xkcd.com/comics/password_strength.png')

### System and network utils

In [ ]:
network_utils_libs = ['fabric', 'sh', 'psutil', 'click', 'scapy', 'scrapy', ]

In [ ]:
@pytest.mark.parametrize(
    "library_name", 
    network_utils_libs
)
def test_network_util_imports(library_name):
    import importlib
    
    importlib.import_module(library_name)

## Running the tests

In [ ]:
ipytest.run()